![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
qb = QuantBook()

splits = qb.add_data(EODHDUpcomingSplits, "splits").symbol

be_tested = qb.history(splits, datetime(2010, 1, 1), datetime(2025, 2, 20), Resolution.DAILY)
cleaned_be_tested = be_tested.reset_index()[["symbol", "splitdate", "splitfactor"]].groupby(["symbol", "splitdate"]).last()
cleaned_be_tested

In [2]:
symbols = list(set(be_tested.index.get_level_values(0)))
benchmark = qb.history(Split, symbols, datetime(2010, 1, 1), datetime(2025, 2, 20), Resolution.DAILY)
cleaned_benchmark = benchmark[benchmark["type"] == 1].reset_index()[["symbol", "time", "splitfactor"]]
cleaned_benchmark.columns = ["symbol", "splitdate", "splitfactor"]
cleaned_benchmark = cleaned_benchmark[cleaned_benchmark["splitdate"] >= datetime(2010, 1, 1)].groupby(["symbol", "splitdate"]).last()
cleaned_benchmark

In [3]:
def create_confusion_maxtrix(cleaned_be_tested, cleaned_benchmark, N, date_since=None):
    if date_since:
        cleaned_be_tested = cleaned_be_tested[cleaned_be_tested.index.get_level_values(1) >= date_since]
        cleaned_benchmark = cleaned_benchmark[cleaned_benchmark.index.get_level_values(1) >= date_since]

    index_to_be_tested = list(cleaned_benchmark.index)
    k = 0
    while k < N:
        index_to_be_tested += list(pd.MultiIndex.from_tuples([(s, i+timedelta(k)) for (s, i) in cleaned_benchmark.index])) + list(pd.MultiIndex.from_tuples([(s, i-timedelta(k)) for (s, i) in cleaned_benchmark.index]))
        k += 1

    tp = cleaned_be_tested[cleaned_be_tested.index.isin(index_to_be_tested)]
    fp = cleaned_be_tested[~cleaned_be_tested.index.isin(index_to_be_tested)]

    index_to_be_tested = list(cleaned_be_tested.index)
    k = 0
    while k < N:
        index_to_be_tested += list(pd.MultiIndex.from_tuples([(s, i+timedelta(k)) for (s, i) in cleaned_be_tested.index])) + list(pd.MultiIndex.from_tuples([(s, i-timedelta(k)) for (s, i) in cleaned_be_tested.index]))
        k += 1

    fn = cleaned_benchmark[~cleaned_benchmark.index.isin(index_to_be_tested)]

    # Create confusion matrix
    # True Negatives (TN) is set to 0
    confusion_matrix = np.array([[tp.shape[0], fn.shape[0]], [fp.shape[0], 0]])
    return confusion_matrix

In [22]:
cm = create_confusion_maxtrix(cleaned_be_tested, cleaned_benchmark, 1)
print(f"±1 days Precision: {cm[0][0]/np.sum(cm[0])}, Missed Rate: {cm[0][1]/np.sum(cm[0])}, False Positive Rate (to be investigated): {cm[1][0]/np.sum(cm[:, 0])}")

In [23]:
cm = create_confusion_maxtrix(cleaned_be_tested, cleaned_benchmark, 3)
print(f"±3 days Precision: {cm[0][0]/np.sum(cm[0])}, Missed Rate: {cm[0][1]/np.sum(cm[0])}, False Positive Rate (to be investigated): {cm[1][0]/np.sum(cm[:, 0])}")

In [24]:
cm = create_confusion_maxtrix(cleaned_be_tested, cleaned_benchmark, 7)
print(f"±7 days Precision: {cm[0][0]/np.sum(cm[0])}, Missed Rate: {cm[0][1]/np.sum(cm[0])}, False Positive Rate (to be investigated): {cm[1][0]/np.sum(cm[:, 0])}")

In [25]:
cm = create_confusion_maxtrix(cleaned_be_tested, cleaned_benchmark, 1, datetime(2015, 1, 1))
print(f"±1 days Precision since 2015: {cm[0][0]/np.sum(cm[0])}, Missed Rate: {cm[0][1]/np.sum(cm[0])}, False Positive Rate (to be investigated): {cm[1][0]/np.sum(cm[:, 0])}")

In [26]:
cm = create_confusion_maxtrix(cleaned_be_tested, cleaned_benchmark, 3, datetime(2015, 1, 1))
print(f"±3 days Precision since 2015: {cm[0][0]/np.sum(cm[0])}, Missed Rate: {cm[0][1]/np.sum(cm[0])}, False Positive Rate (to be investigated): {cm[1][0]/np.sum(cm[:, 0])}")

In [27]:
cm = create_confusion_maxtrix(cleaned_be_tested, cleaned_benchmark, 7, datetime(2015, 1, 1))
print(f"±7 days Precision since 2015: {cm[0][0]/np.sum(cm[0])}, Missed Rate: {cm[0][1]/np.sum(cm[0])}, False Positive Rate (to be investigated): {cm[1][0]/np.sum(cm[:, 0])}")